In [1]:
from pymilvus import connections, Collection, utility
from pymilvus import utility
import google.generativeai as genai
from local_settings import API_KEY,uri,token
from pymilvus import MilvusClient
from helper_func import create_prompt, process_data

connections.connect(uri=uri,token=token)
milvus_client = MilvusClient(uri=uri,token=token)

collection_name = "my_rag_collection2"

In [2]:

collection = Collection(collection_name)
collection.num_entities 

10663

In [3]:
utility.list_collections()

['my_rag_collection2']

In [4]:
res = collection.query(expr="", output_fields=["count(*)"])
print(res)

[{'count(*)': 10663}]


In [5]:
embedding_model = 'models/text-embedding-004'
genai.configure(api_key=API_KEY)
def embed_fn(title, text):
  return genai.embed_content(model=embedding_model,
                             content=text,
                             task_type="retrieval_document",
                             title=title)["embedding"]

In [6]:
question = "Who Is A Key Employee In 401K?"

search_res = milvus_client.search(
    collection_name=collection_name,
    data=[
        embed_fn(question,question)
    ],  # Use the `emb_text` function to convert the question to an embedding vector
    limit=7,  # Return top 3 results
    search_params={"metric_type": "COSINE",  "params": {"level": 2}},  # Inner product distance
    output_fields=["text"],  # Return the text field
)

In [7]:
retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
retrieved_lines_with_distances

[(['Who Is A Key Employee In 401K? ',
   'An employer can establish a defined contribution retirement plan in accordance with Internal Revenue Code 401 -LRB- k. -RRB- The employer is responsible for the plan documents including definitions . While not required there may be some mention of key employees . If your plan has provisions like this , they would be in the plan document which is available from your employer . '],
  0.8981134295463562),
 (['What Kind Of Retirement Plan Is A 401K? ',
   'The Internal Revenue Code -LRB- IRC -RRB- allows employers to establish defined contribution pension plans under section 401 -LRB- k -RRB- in these plans the assets are segregated into individual employee accounts . Often the employer will offer a range of investment options to the participant . The results of the account are dependent upon the investments selected . Some employers contribute to the plans , some do not . '],
  0.722854733467102),
 (['Who Governs 401K Plans? ',
   'A defined contr

In [8]:
querry_data= process_data(retrieved_lines_with_distances)
print(querry_data)

Q: Who Is A Key Employee In 401K?
A: An employer can establish a defined contribution retirement plan in accordance with Internal Revenue Code 401 -LRB- k. -RRB- The employer is responsible for the plan documents including definitions . While not required there may be some mention of key employees . If your plan has provisions like this , they would be in the plan document which is available from your employer .

Q: What Kind Of Retirement Plan Is A 401K?
A: The Internal Revenue Code -LRB- IRC -RRB- allows employers to establish defined contribution pension plans under section 401 -LRB- k -RRB- in these plans the assets are segregated into individual employee accounts . Often the employer will offer a range of investment options to the participant . The results of the account are dependent upon the investments selected . Some employers contribute to the plans , some do not .

Q: Who Governs 401K Plans?
A: A defined contribution pension plan established by an employer under section 401 

In [21]:
testing_question="What is so special about LLamba 2"
prompt=create_prompt(querry_data,testing_question)
print(prompt)


Task:Your are a Insurance assistance, your job is based on a conversation of a costumer and a insurance expert, 
answer insurance-related questions accurately and provide some short explanation.
Instructions:
Read the context carefully.
Consider key points, concepts, and details within the document.
Generate questions that seek information or clarification about the document's content.
Ensure that the questions are grammatically correct and understandable.
Give the answer directly, do not repeat the question or add any unrelated text
If user talk to you casually, reply normally
If the question is not related to the context or there is no answer in the context, tell the users the question is not related to insurance or there is no answer in your database
Context:
Q: Who Is A Key Employee In 401K?
A: An employer can establish a defined contribution retirement plan in accordance with Internal Revenue Code 401 -LRB- k. -RRB- The employer is responsible for the plan documents including def

In [22]:
LLM_model = genai.GenerativeModel('gemini-pro')
response = LLM_model.generate_content(prompt)

In [23]:
print( response.text.replace("JSON", "").replace("```",""))

The question is not related to insurance
